In [28]:
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm   # Acrescentado tqdm para barra de progresso
from calendar import monthrange # Cria uma lista de tuplas (mês, dia) apenas para datas válidas

In [ ]:
def ler_csv(arquivo_csv, cod_estacao, codificacao='latin1'):
    """Lê o csv e converte a coluna de data para datetime.
    Vazões totais = latin1
    FLU_Series_ANA = utf-8
    """
    parse_dates = None

    # Detectar o nome correto da coluna de data (pode vir com aspas ou não)
    # Apenas lê a primeira linha para inspecionar os nomes das colunas
    if arquivo_csv.endswith('.csv'):
        colunas = pd.read_csv(arquivo_csv, encoding=codificacao, engine='python', nrows=0).columns
    elif arquivo_csv.endswith('.txt'):
        colunas = pd.read_csv(arquivo_csv, sep='\\t', encoding=codificacao, engine='python', nrows=0).columns
    else:
        raise ValueError("Arquivo deve ser .csv ou .txt")
    
    # Normalizar nomes para comparar
    colunas_norm = [col.strip().replace('"', '') for col in colunas]
    if "Data" in colunas_norm:
        parse_dates = [colunas[list(colunas_norm).index("Data")]]

    # Agora lê o arquivo já com parse_dates se detectou a coluna
    if arquivo_csv.endswith('.csv'):
        df = pd.read_csv(arquivo_csv, engine='python', parse_dates=parse_dates)
    elif arquivo_csv.endswith('.txt'):
        df = pd.read_csv(arquivo_csv, sep='\\t', encoding='latin1', engine='python', parse_dates=parse_dates)
    
    # Corrigir nomes das colunas para remover aspas e facilitar o acesso
    df.columns = [col.strip().replace('"', '') for col in df.columns]
        
    if "Cod.estacao" in df.columns:
        df.rename(columns={"Cod.estacao": "cod_estacao"}, inplace=True)
    
    if "Cod_estacao" in df.columns:
        df.rename(columns={"Cod_estacao": "cod_estacao"}, inplace=True)
        
    if "cod_estacao" not in df.columns:
        raise ValueError("A coluna 'cod_estacao' não foi encontrada no arquivo.")
    
    df = df[df["cod_estacao"] == cod_estacao]
    
    # Corrigir coluna de data: remover aspas e converter para datetime
    if "Data" in df.columns:
        df["Data"] = df["Data"].astype(str).str.replace('"', '').str.strip()
        df["Data"] = pd.to_datetime(df["Data"], format="%Y-%m-%d", errors="coerce")
        
    return df

In [30]:
def criar_dia_mes_ano(df):
    df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d', errors='coerce')
    df['Dia'] = df['Data'].dt.day
    df['Mes'] = df['Data'].dt.month
    df['Ano'] = df['Data'].dt.year
    return df

In [31]:
def filtrar_anos_incompletos(df):
    # Filtra anos que não possuem todos os meses
    anos_completos = df['Ano'].value_counts()[df['Ano'].value_counts() >= 365].index
    return df[df['Ano'].isin(anos_completos)]

In [32]:
def criar_vazao_media_diaria_por_mes(df):
    
    resultados = []
    for m in range(1, 13):
        dias_no_mes = monthrange(2000, m)[1]  # 2000 é um ano bissexto, mas só importa o número de dias
        for d in range(1, dias_no_mes + 1):
            df_filtro = df[(df['Mes'] == m) & (df['Dia'] == d)]
            media = round(df_filtro['Vazao'].mean(), 2)
            resultados.append({'Mes': m, 'Dia': d, 'Vazao': media})

    df_resultados = pd.DataFrame(resultados)
    return df_resultados

In [33]:
def criar_lista_anos_serie_indicar_bissexto(ano_inicial, ano_final):
    
    # Criar lista de anos da série
    lista_anos_serie = list(range(ano_inicial, ano_final + 1))
    
    # Listar anos bissextos 
    anos_bissextos = [ano for ano in lista_anos_serie if (ano % 4 == 0 and ano % 100 != 0) or (ano % 400 == 0)]

    # df 
    df_anos = pd.DataFrame({
    'Ano': lista_anos_serie,
    'Bissexto': [1 if ano in anos_bissextos else 0 for ano in lista_anos_serie]
    })
    return df_anos


In [34]:
def criar_arquivo_estacao_virtual_txt(id, nome, lat, long, elevation, nome_arquivo):
    id = id
    name = nome
    lat = lat
    long = long
    elevation = elevation
        
    df = pd.DataFrame({
            'ID': [id],
            'NAME': [name],
            'LAT': [lat],
            'LONG': [long],
            'ELEVATION': [elevation]
        })
        
        # Criar o a rquivo de saída
    with open(f"{os.path.join(os.getcwd(), nome_arquivo)}.txt", 'w') as f:
        f.write("ID,NAME,LAT,LONG,ELEVATION\n")
        # Se tiver mais de uma linha, escrever todas
        if len(df) > 1:
            for index, row in df.iterrows():
                    f.write(f"{row['ID']},{row['NAME']},{row['LAT']},{row['LONG']},{row['ELEVATION']}\n")
        else:
            # Se tiver apenas uma linha, escrever apenas uma vez
            f.write(f"{df['ID'][0]},{df['NAME'][0]},{df['LAT'][0]},{df['LONG'][0]},{df['ELEVATION'][0]}\n")
        print(f"Arquivo '{nome_arquivo}.txt' criado com sucesso.")

In [35]:
def criar_arquivo_dados_estacao_virtual_txt(df, var_mes, var_ano, var_vazao, nome_arquivo, subbacia ):
    """Cria um arquivo de dados climáticos formatado."""
                
    with open(f"{nome_arquivo}_{subbacia}.txt", 'w') as f:
        f.write('"Month","Year","Flomon","Sedmon","Orgnmon","Orgpmon","No3mon","Nh3mon","No2mon","Minpmon","Cbodmon","Disoxmon","Chlamon","Solpstmon","Srbpstmon","Bactpmon","Bactlpmon","Cmtl1mon","Cmtl2mon","Cmtl3mon"\n')
        
        for _, row in df.iterrows():
            mes = int(row[var_mes])
            ano = int(row[var_ano])
            vazao = round(row[var_vazao],3)
            
            f.write(f"{mes},{ano},{vazao},0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000\n")
        print(f"Arquivo '{nome_arquivo}_{subbacia}' criado com sucesso.")
        

In [ ]:
print("Iniciando pipeline de precipitação...")
media_diaria = False
lista_cod_estacao = [60473000, 60471185, 60474000]
subbacias = [40, 15, 34]
arquivo_csv = "FLU_Series_ANA.txt"
# lats = [-15.8094,-15.8186, -15.8186, -15.7956, -15.7925]
# longs = [-47.7006,-47.705, -47.7047, -47.7844, -47.7647]
# alts = [773,860,860,1000, 885.18]
# nome_arquivo_est = "vazao"
# for i, estacao, lat, long, alt in zip(range(len(lista_cod_estacao)), lista_cod_estacao, lats, longs, alts):
#     criar_arquivo_estacao_virtual_txt(i + 1, f"v{estacao}", lat, long, alt, nome_arquivo_est)

for estacao, subbacia in tqdm(zip(lista_cod_estacao, subbacias), total=len(lista_cod_estacao), desc="Processando estações"):
    df = ler_csv(arquivo_csv, estacao)
    df = criar_dia_mes_ano(df)
    df = filtrar_anos_incompletos(df)
    
    if media_diaria:
        df = criar_vazao_media_diaria_por_mes(df)

    df_anos = criar_lista_anos_serie_indicar_bissexto(1966, 2009)
    
    # Corrigir: manter como DataFrame com colunas 'Mes' e 'Vazao'
    df_ano_normal = df[~((df['Mes'] == 2) & (df['Dia'] == 29))].groupby(['Mes'])['Vazao'].mean().round(3).reset_index()
    df_ano_normal['Vazao'] = df_ano_normal['Vazao'] * 60 * 60 * 24
    
    df_ano_bissexto = df.groupby(['Mes'])['Vazao'].mean().round(3).reset_index()
    df_ano_bissexto['Vazao'] = df_ano_bissexto['Vazao'] * 60 * 60 * 24
    
    # Criar um dataframe com os anos e os dados de vazão por mes
    lista = []
    for _, row in df_anos.iterrows():
        ano = row['Ano']
        bissexto = row['Bissexto']
        if bissexto:
            for _, mes_row in df_ano_bissexto.iterrows():
                lista.append({'Ano': ano, 'Mes': int(mes_row['Mes']), 'Vazao': mes_row['Vazao']})
        else:
            for _, mes_row in df_ano_normal.iterrows():
                lista.append({'Ano': ano, 'Mes': int(mes_row['Mes']), 'Vazao': mes_row['Vazao']})

    df_serie_anos = pd.DataFrame(lista)

    criar_arquivo_dados_estacao_virtual_txt(df_serie_anos, 'Mes', 'Ano', 'Vazao', estacao, subbacia)

Iniciando pipeline de precipitação...


Processando estações:  10%|█         | 1/10 [00:01<00:13,  1.55s/it]

Arquivo '60476155_62' criado com sucesso.


Processando estações:  20%|██        | 2/10 [00:03<00:12,  1.55s/it]

Arquivo '60476160_20' criado com sucesso.


...

Arquivo '60480200_61' criado com sucesso.


Processando estações:  90%|█████████ | 9/10 [00:14<00:01,  1.58s/it]

Arquivo '60480310_51' criado com sucesso.


Processando estações: 100%|██████████| 10/10 [00:16<00:00,  1.63s/it]

Arquivo '60480550_71' criado com sucesso.
